#### Code to calculate the mass budget from the input dh raster and the elavation raster
#### It calculates the weighted mean of dh for each 50 m altitude interval and then calculates the mass budget using a constant density scenario..
#### can easily be tweaked for the dual density scenario (just need to split at the ELA (Eq. Line Altitutde if information on it is available)

In [2]:
#libraries to be used for evaluation
import numpy as np
import numpy.ma as ma
from pygeotools.lib import malib, warplib, iolib, geolib
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
#importing rasters after warping them through GDAL Warp as masked arrays to common extent
fn1='Elevation.tif' #DEM Clipped to glacier extent
fn2='dh_glaciers.tif' #Elevation Difference Clipped to Glacier extent
ds_list=warplib.memwarp_multi_fn([fn1,fn2],res='max',extent='intersection',t_srs='first', r='cubic')
A=iolib.ds_getma(ds_list[0])
dh=iolib.ds_getma(ds_list[1])
E=ma.asarray(A,dtype=np.float32)


Warping all inputs to the following:
Resolution: 30.0
Extent: [594711.267, 3697416.167, 631071.267, 3753786.167]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: cubic

1 of 2: Elevation.tif
2 of 2: dh_glaciers.tif


In [4]:
#Compressing the masked arrays as 1 dimensional arrays containing only the unmasked valid pixels
Dh_Required=ma.compressed(dh)
Elevation_R=ma.compressed(E)
raw_data={'Elevation':Elevation_R,
         'dh':Dh_Required}
df=pd.DataFrame(raw_data,columns=['Elevation','dh'])

In [5]:
#Elevation_rounded to next multiple of 50
#ECW=(E_Required + 49) // 50 * 50
df['Elevation_rounded'] = (df['Elevation']+49)//50*50

In [6]:
df

,Elevation,dh,Elevation_rounded
0,4547.0,1.278813,4550.0
1,4574.0,-12.724151,4600.0
2,4587.0,-14.727103,4600.0
3,4609.0,-24.730045,4650.0
4,4629.0,-22.732975,4650.0
5,4648.0,-26.735893,4650.0
6,4669.0,-28.738802,4700.0
7,4686.0,-17.741697,4700.0
8,4701.0,-15.744583,4750.0
9,4710.0,8.252543,4750.0


#### Grouping based on the rounded elevation
#### This is a hack based on the assumption that the rounded elevation value will be taken by each of the pixels in that previous 50 m altitude range

In [6]:
grouped=df.groupby('Elevation_rounded')
df1=grouped['dh'].agg([np.count_nonzero,np.sum, np.mean, np.std])
df1

,count_nonzero,sum,mean,std
Elevation_rounded,,,,
3550.0,83.0,15.213154,0.183291,6.283049
3600.0,459.0,-4188.076172,-9.124349,9.402762
3650.0,1655.0,-29696.236328,-17.943346,15.289424
3700.0,1685.0,-23961.699219,-14.220593,13.216174
3750.0,2178.0,-31892.255859,-14.642909,12.554529
3800.0,2863.0,-47649.761719,-16.643297,15.161282
3850.0,3351.0,-37328.167969,-11.139412,18.917442
3900.0,4454.0,-52712.425781,-11.834850,11.490685
3950.0,5012.0,-64596.136719,-12.888296,11.685396


#### Maths for the Calculation
#### Each interval mean is multiplied with the no of pixels in the interval and the summation of this step for all the intervals is divided by the total pixel count to get the net elevation difference 

In [7]:
df1['mean*count']=df1['mean']*df1['count_nonzero']
df1

,count_nonzero,sum,mean,std,mean*count
Elevation_rounded,,,,,
3550.0,83.0,15.213154,0.183291,6.283049,15.213155
3600.0,459.0,-4188.076172,-9.124349,9.402762,-4188.076172
3650.0,1655.0,-29696.236328,-17.943346,15.289424,-29696.238281
3700.0,1685.0,-23961.699219,-14.220593,13.216174,-23961.699219
3750.0,2178.0,-31892.255859,-14.642909,12.554529,-31892.255859
3800.0,2863.0,-47649.761719,-16.643297,15.161282,-47649.761719
3850.0,3351.0,-37328.167969,-11.139412,18.917442,-37328.167969
3900.0,4454.0,-52712.425781,-11.834850,11.490685,-52712.421875
3950.0,5012.0,-64596.136719,-12.888296,11.685396,-64596.140625


In [8]:
Sum_of_mean_count=df1['mean*count'].sum()
Sum_of_all_pixels=df1['count_nonzero'].sum()
Net_Elevation_difference=Sum_of_mean_count/Sum_of_all_pixels
print 'Net elevation difference is %s m' %str(Net_Elevation_difference)

Net elevation difference is -1.53687 m


In [10]:
src_ds = iolib.fn_getds(fn1)
res = geolib.get_res(src_ds, square=True)
res

[30.0, 30.0]

In [11]:
A=Sum_of_all_pixels*res[0]*res[0]
total_dh=df['dh'].sum()
dv=Net_Elevation_difference*A
print 'Volume_change in m_cube= %s' %str(dv)
mass_budget=Net_Elevation_difference*0.850
print 'Mass_budget in m w.eq. = %s' %str(mass_budget)


Volume_change in m_cube= -898045057.871
Mass_budget in m w.eq. = -1.30634102225
